This Notebook is for analyzing profile test data. The test data includes 20 json files. This notebook will load the data into dataframe and analyzed using Spark SQL.

1. Import PySpark Library

In [1]:
from pyspark.sql import SparkSession

2. Create SparkSession

In [2]:
spark = SparkSession \
    .builder \
    .appName("Profile Analyse") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()